In [ ]:
# !pip install spacy==3.1.4
# !pip install spacy-transformers
# !python -m spacy download en_core_web_trf

In [ ]:
import spacy
import pandas as pd
import matplotlib.pyplot as plt
import torch
import numpy as np

# Transformers with SpaCy
https://spacy.io/models/en#en_core_web_trf

In [ ]:
nlp = spacy.load("en_core_web_trf")

In [ ]:
query = nlp("happy white woman") 
doc1 = nlp("Happy Young  Woman In White Dress Walking In Spring Park in New York...")
doc2 = nlp("Happy Couple White Woman Black Man Stock Photo (Edit Now) 47205217")

In [ ]:
doc1._.trf_data.tokens

#### Word vectors

In [ ]:
query._.trf_data.tensors

In [ ]:
query._.trf_data.tensors[0].shape

In [ ]:
dataset = pd.read_csv('data/dataset.csv')
dataset = dataset[dataset['query'] == 'happy+white+woman']

In [ ]:
def get_word_vector(text, word):
    doc = nlp(text)
    vector = [0]*768
    for i, token in enumerate(doc._.trf_data.tokens['input_texts'][0]):
        if word in token.lower():
            vector = doc._.trf_data.tensors[0][0][i]
    return vector

In [ ]:
dataset['vector white'] = dataset['title'].apply(get_word_vector, word='white')

## Cosine similarity
https://deepai.org/machine-learning-glossary-and-terms/cosine-similarity

In [ ]:
def cos_sim(v1, v2):
    return np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))

In [ ]:
query_vector = get_word_vector('happy white woman', 'white')

In [ ]:
dataset['similarity'] = dataset['vector white'].apply(cos_sim, v2=query_vector)

google_dataset = dataset[dataset['engine']=='google'][:30]
bing_dataset = dataset[dataset['engine']=='bing'][:30]

In [ ]:
google_dataset.head()

In [ ]:
bing_dataset.head()

#### Plot results

In [ ]:
plt.scatter(bing_dataset['baseline'], bing_dataset['similarity'])
plt.scatter(google_dataset['baseline'], google_dataset['similarity'])
plt.legend(['bing', 'google'])
plt.xlabel("baseline")
plt.ylabel("similarity")
plt.show()

In [ ]:
def print_summary(bing_data, google_data, query):
    print('Similarity for word vectors')
    print("\tMean similarity for Google results (top 5): ", google_data['similarity'][:5].mean())
    print("\tMean similarity for Bing results (top 5): ", bing_data['similarity'][:5].mean())
    print()
    print("\tMean similarity for Google results: ", google_data['similarity'].mean())
    print("\tMean similarity for Bing results: ", bing_data['similarity'].mean())

    
print_summary(bing_dataset, google_dataset, 'happy white woman')

## PCA
https://builtin.com/data-science/step-step-explanation-principal-component-analysis

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
pca = PCA(n_components=2)

standardized = StandardScaler().fit_transform(dataset['vector white'].tolist())
dataset[['PC 1', 'PC 2']] = pca.fit_transform(standardized)

dataset.head()

In [ ]:
google_dataset = dataset[dataset['engine']=='google']
bing_dataset = dataset[dataset['engine']=='bing']

plt.scatter(bing_dataset['PC 1'], bing_dataset['PC 2'])
plt.scatter(google_dataset['PC 1'], google_dataset['PC 2'])
plt.legend(['bing', 'google'])
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.show()